# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We loaded more information to the Search Engine this time using a CSV file with 90k rows/articles in index: "cogsrch-index-csv"
- **Notebook 03**: We added AzureOpenAI GPT models to enhance the the production of the answer by using Utility Chains of LLMs
- **Notebook 04**: We manually loaded an index with large/complex PDFs information , "cogsrch-index-books-vector"
- **Notebook 05**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 06**: We introduced Agents and Tools and built the first Skill/Agent, that can do RAG over a search engine
- **Notebook 07**: We build a second Agent (Pandas) in order to be able to solve a more complex task: ask questions to Tabular datasets
- **Notebook 08**: We used a SQL Agent in order to talk to a SQL Database directly
- **Notebook 09**: We used another  Agent in order to talk to the Bing Search API and create a Bing Chat Clone and implemented callbacks for real-time streaming and tool information
- **Notebook 10**: We built an API Agent that can translate a question into the right API calls, giving us the capability to talk to any datasource that provides a RESTFul API.


We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, APIs, etc), the Assistant can answer the question correctly using the right tool.

In this Notebook we are going to create that "brain" Agent (also called Master Agent), that:

1) understands the question, interacts with the user 
2) talks to other specialized Agents that are connected to diferent sources
3) once it get's the answer it delivers it to the user or let the specialized Agent to deliver it directly

This is the same concept of [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agents talking to each other.

![image](https://www.microsoft.com/en-us/research/uploads/prod/2023/09/AutoGen_Fig1.png)

In [1]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent 
    #SQLSearchAgent, 
    #ChatGPTTool, 
    #BingSearchAgent, 
    #APISearchAgent, 
    #reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [2]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Get the Tools - DocSearch Agent, CSV Agent, SQL Agent, Web Search Agent, ChatGPT, API Agent

**Consider the following concept:** Agents, which are essentially software entities designed to perform specific tasks, can be equipped with tools. These tools themselves can be other agents, each possessing their own set of tools. This creates a layered structure where tools can range from code sequences to human actions, forming interconnected chains. Ultimately, you're constructing a network of agents and their respective tools, all collaboratively working towards solving a specific task (This is what ChatGPT is). This network operates by leveraging the unique capabilities of each agent and tool, creating a dynamic and efficient system for task resolution.

In the file `common/utils.py` we created Agent Tools Classes for each of the Functionalities that we developed in prior Notebooks. 

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


In [ ]:
doc_indexes = ["srch-index-files", "srch-index-csv"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           callback_manager=cb_manager, verbose=False)

In [ ]:
book_indexes = ["srch-index-books"]
book_search = DocSearchAgent(llm=llm, indexes=book_indexes,
                           k=10, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="booksearch",
                           description="useful when the questions includes the term: booksearch",
                           callback_manager=cb_manager, verbose=False)

In [ ]:
# BingSearchAgent is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchAgent(llm=llm, k=5, callback_manager=cb_manager, 
                             name="bing",
                             description="useful when the questions includes the term: bing",
                             verbose=False)

In [4]:
## CSVTabularAgent is a custom Tool class crated to Q&A over CSV files
file_url = "./data/TCK.csv"
csv_search_TCK = CSVTabularAgent(path=file_url, llm=llm, callback_manager=cb_manager,
                             name="csv_TCK",
                             description="useful when the questions includes the term: TICKET",
                             verbose=False)

file_url = "./data/SF.csv"
csv_search_SF = CSVTabularAgent(path=file_url, llm=llm, callback_manager=cb_manager,
                             name="csv_SF",
                             description="useful when the questions includes the term: csv_SF",
                             verbose=False)

file_url = "./data/CMDB.csv"
csv_search_CMDB = CSVTabularAgent(path=file_url, llm=llm, callback_manager=cb_manager,
                             name="csv_CMDB",
                             description="useful when the questions includes the term: csv_CMDB",
                             verbose=False)

In [ ]:
## SQLDbAgent is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLSearchAgent(llm=llm, k=30, callback_manager=cb_manager,
                            name="sqlsearch",
                            description="useful when the questions includes the term: sqlsearch",
                            verbose=False)

In [ ]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager,
                             name="chatgpt",
                            description="useful when the questions includes the term: chatgpt",
                            verbose=False)

In [ ]:
## APISearchAgent is a custom Tool class created to talk to any API 

url = "https://datasetsgptsmartsearch.blob.core.windows.net/apispecs/openapi_kraken.json"
spec = requests.get(url + os.environ['BLOB_SAS_TOKEN']).json()

api_search = APISearchAgent(llm=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            llm_search=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            api_spec=str(reduce_openapi_spec(spec)),
                            callback_manager=cb_manager,
                            name="apisearch",
                            description="useful when the questions includes the term: apisearch",
                            verbose=False)

### Variables/knobs to use for customization

As you have seen so far, there are many knobs that you can dial up or down in order to change the behavior of your GPT Smart Search engine application, these are the variables you can tune:

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. For a production system you will need gpt-4-32k. This is the model that will give you enough reasoning power to work with agents, and enough tokens to work with detailed answers and conversation memory.
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchAgent</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
- <u>BingSearchAgent</u>:
  - **k**: The top k results from the bing search action
- <u>SQLSearchAgent</u>:
  - **k**: The top k results from the SQL search action. Adds TOP clause to the query
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [ ]:
# Test the Documents Search Tool with a question we know it doesn't have the knowledge for
printmd(doc_search.run("what is the weather today in Dallas?"))

In [ ]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(await doc_search.arun("How Covid affects obese people? and elderly?"))

In [ ]:
# Test the other index created manually
printmd(await book_search.arun("Tell me about the kidney stolen legend?"))

In [ ]:
# Test the Bing Search Agent
printmd(await www_search.arun("Who are the family member names of the current president of India?"))

In [ ]:
# Test the CSV Agent
printmd(await csv_search_CMDB.arun("de cuantos clientes distintos tienes información?"))

In [ ]:
# Test the SQL Search Agent
printmd(await sql_search.arun("How many people in total died california in each state of the west coast in July 2020?"))

In [ ]:
# Test the ChatGPTWrapper Search Tool
printmd(await chatgpt_search.arun("what is the function in python that allows me to get a random number?"))

In [ ]:
# Test the API Search Tool - This will be slower since it is using GPT-4
printmd(await api_search.arun("what is the price now of Bitcoin? and of Ethereum?"))

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [5]:
#tools = [www_search, sql_search, doc_search, book_search, chatgpt_search, csv_search]
tools = [csv_search_TCK, csv_search_SF, csv_search_CMDB]

**Note**: Notice that since both the CSV file and the SQL Database have the same exact data, we are only going to use the SQLDBTool since it is faster and more reliable

# Option 1: Using OpenAI functions as router

We need a method to route the question to the right tool, one simple way to do this is to use OpenAI models functions via the Tools API (models 1106 and newer). To do this, we need to bind these tools/functions to the model and let the model respond with the right tool to use.

The advantage of this option is that there is no another agent in the middle between the experts (agent tools) and the user. Each agent tool responds directly. Also, another advantage is that multiple tools can be called in parallel.

**Note**: on this method it is important that each agent tool has the same system profile prompt so they adhere to the same reponse guidelines.

In [6]:
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

In [ ]:
def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["type"]]
    return RunnablePassthrough.assign(output=itemgetter("args") | tool)

def print_response(result: List):
    for answer in result:
        printmd("**"+answer["type"] + "**" + ": " + answer["output"])
        printmd("----")
    
# .map() allows us to apply a function to a list of inputs.
call_tool_list = RunnableLambda(call_tool).map()
agent = llm_with_tools | JsonOutputToolsParser() | call_tool_list

In [ ]:
result = agent.invoke("Who is the current president of France?")
print_response(result)

In [ ]:
result = agent.invoke("csvfile, cuantas filas tienes?")
print_response(result)

# Option 2: Using a user facing agent that calls the agent tools experts

With this method, we create a user facing agent that talks to the user and also talks to the experts (agent tools)

### Initialize the brain agent

In [7]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [8]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [9]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [10]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [17]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

session237 user957


### Let's talk to our GPT Smart Search Engine chat bot now

In [ ]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(brain_agent_executor.invoke({"question": "Hi, I'm Pablo Marin, how are you doing today?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "what is your name and what do you do?"}, config=config)["output"])

In [18]:
printmd(brain_agent_executor.invoke({"question": "csv_SF, cual es el TCV total del cliente Toka?"}, 
                                    config=config)["output"])

Parent run a989fce7-2d3c-4811-97a5-d12ebf9accb5 not found for run 3b622074-f23e-4930-b38e-bc5e07d81157. Treating as a root run.


Tool: csv_SF
Agent Action: 
Invoking: `python_repl_ast` with `{'query': "import pandas as pd\npd.set_option('display.max_columns', None)\ndf.head()"}`



Agent Action: 
Invoking: `python_repl_ast` with `{'query': 'df.columns'}`



Agent Action: 
Invoking: `python_repl_ast` with `{'query': "# Remove extra spaces from column names\ndf.columns = df.columns.str.strip()\n# Filter rows where 'cliente' is 'Toka'\ntoka_df = df[df['cliente'] == 'Toka']\n# Convert 'TCV' to numeric after removing '$' and commas\ntoka_df['TCV'] = toka_df['TCV'].replace({'[$,]': ''}, regex=True).astype(float)\n# Sum the 'TCV' values\ntoka_tcv_total = toka_df['TCV'].sum()\ntoka_tcv_total"}`



Agent Action: 
Invoking: `python_repl_ast` with `{'query': "# Check unique values in the 'cliente' column to ensure correct filtering\nunique_clients = df['cliente'].unique()\nunique_clients"}`





### TCV Total del Cliente Toka

Después de revisar los datos proporcionados, he verificado que no hay registros asociados con el cliente "Toka". Los clientes presentes en los datos son "Abilia" y "Bulkmatic".

#### Explicación:
1. **Extracción de Columnas**: Primero, obtuve los nombres de las columnas y eliminé los espacios adicionales.
2. **Filtrado de Datos**: Filtré las filas donde el cliente es "Toka".
3. **Conversión de Valores**: Convertí los valores de la columna `TCV` a tipo numérico después de eliminar los caracteres `$` y `,`.
4. **Suma de TCV**: Sumé los valores de `TCV` para obtener el total.

Sin embargo, al revisar los valores únicos en la columna `cliente`, no se encontró ningún registro para "Toka".

Por lo tanto, el total de TCV para el cliente "Toka" es **0**.

```plaintext
No se encontraron registros para el cliente "Toka".
```

### Clientes Presentes en los Datos:
- Abilia
- Bulkmatic

In [ ]:
printmd(brain_agent_executor.invoke({"question": "de esta información dame el TCV total de solo aquellas que su Fecha de vigencia sea posterior a hoy?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "cuantas filas tienes?"}, config=config)["output"])

In [12]:
printmd(brain_agent_executor.invoke({"question": "csv_SF, csv_CMDB, csv_TCK, Del cliente Abilia dime cual es su TCV total, cuantos tickets tiene, y cuantas tecnologias se administran?"}, config=config)["output"])

Parent run 2e673aae-4bc9-435a-b09a-81b26f7adac9 not found for run 275684ae-90f5-4adb-a810-5ee7b7b559fe. Treating as a root run.


Tool: csv_SF
Agent Action: 
Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nfrom io import StringIO\n\ndata = '''cliente   | Etapa      | Tipo de Negocio   | Fecha de cierre   | Fecha Fin Contrato   | TCV            | Propietario de oportunidad: Nombre completo   | Quote Number   |   Tiempo de contrato | QuoteLine   | Nombre del producto                                                                     | DescripciÃ³n                                                                                                                                 | Tipo Cargo (Venta)   | MRC         | Fabrica                 | Sector                     | No. de proyecto   \nAbilia    | Closed won | Up Selling        | 13-abr-2020       | 13-abr-2022          | $ 32,834.10    | Berenice Altamirano CalderÃ³n (BAJA)          | Q-19698        |                   24 | QL-0316923  | Licencia Trend Micro EndPoint Security                                                  | Licencia Trend Micro

### Información del Cliente "Abilia"

#### Total Contract Value (TCV)
El **TCV total** para el cliente Abilia es de **$32,834.10**.

- **Fecha de cierre**: 13-abr-2020
- **Fecha Fin Contrato**: 13-abr-2022
- **Propietario de oportunidad**: Berenice Altamirano Calderón
- **Número de proyecto**: P-012660

#### Tickets
El cliente Abilia tiene **5 tickets** registrados. A continuación se detallan algunos de ellos:

1. **ID de la solicitud**: 27208
   - **Asunto**: Notificación de FW_ABILIA_Bosque6060 en Estado...
   - **Técnico**: Nestor Severiano Perales
   - **Hora de creación**: 18/02/2023 16:26
   - **Hora de finalización**: 26/02/2023 19:31
   - **Estado de solicitud**: Cerrado

2. **ID de la solicitud**: 32935
   - **Asunto**: (ABILIA) Validación de conexión
   - **Técnico**: Uriel Lopez Martinez
   - **Hora de creación**: 21/02/2023 11:13
   - **Hora de finalización**: 24/02/2023 10:41
   - **Estado de solicitud**: Cerrado

3. **ID de la solicitud**: 35191
   - **Asunto**: (ABILIA) Validación de conexión
   - **Técnico**: Hugo Alvarez Fuentes
   - **Hora de creación**: 22/02/2023 09:42
   - **Hora de finalización**: 24/02/2023 10:42
   - **Estado de solicitud**: Cerrado

4. **ID de la solicitud**: 50010
   - **Asunto**: Validación de automatización de respaldo hacia...
   - **Técnico**: Hugo Alvarez Fuentes
   - **Hora de creación**: 28/02/2023 18:08
   - **Hora de finalización**: 03/03/2023 20:15
   - **Estado de solicitud**: Cerrado

5. **ID de la solicitud**: 60824
   - **Asunto**: Reporte mensual Febrero /// ABILIA
   - **Técnico**: Daniela Yuritzi Torres Alarcon
   - **Hora de creación**: 05/03/2023 04:01
   - **Hora de finalización**: 08/03/2023 04:51
   - **Estado de solicitud**: Cerrado

#### Tecnologías Administradas
El cliente Abilia administra **2 tecnologías** principales:

1. **FIREWALL**
   - **Marca**: FORTINET
   - **Modelo**: FortiGate 300D
   - **Sistema Operativo**: v7.2.5 build1517
   - **Número de serie**: FG100FTK20017803

2. **IPS**
   - **Marca**: FORTINET
   - **Modelo**: FortiWiFi 60D
   - **Sistema Operativo**: v5.4.4, build1117 (GA)
   - **Número de serie**: FWF60D4Q16013527

### Referencias:
1. **Datos de TCV y contratos**:
   - Fuente: [Datos SF](#)
2. **Datos de tickets**:
   - Fuente: [Datos TCK](#)
3. **Datos de tecnologías**:
   - Fuente: [Datos CMDB](#)

In [ ]:
printmd(brain_agent_executor.invoke({"question": "can you tell an example?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "sqlsearch, How many people died of covid in Texas in 2020?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "that result doesn't seem correct, can you use the deathIncrease column instead?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "booksearch, I don't know how to say No to my kids, help me! What kind of boundaries should I set?"}, config=config)["output"])

In [ ]:
# This question although does not contain instructions for a tool, the brain agent decides what tool to use
printmd(brain_agent_executor.invoke({"question": "What's a good place to dine today in downtown Seoul?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "chatgpt, can you give me a javascript example of how to trim the spaces of a sentence?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "Thank you Jarvis!"}, config=config)["output"])

# Option 3: Using LangGraph
See Notebook 11.5

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/docs/integrations/tools/)

# NEXT
It is time now to use all the functions and prompts build so far and build a Web application.
The Next notebook will guide you on how to build:

1) A Bot API Backend
2) A Frontend UI with a Search and Webchat interfaces